<a href="https://colab.research.google.com/github/mgmills93/AEM_Spring_2019/blob/master/SecureCommunities_FinalProject/Dataframe_Creation/FinalProject_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Question 5:	What variables are correlated with SC deportations and how?  

To create this dataset I will draw upon data I have used for previous questions (namely Border and Percent_Hispanic) and add variables per_gop and Incarceration_Number to measure the political stance of counties and the number of incarcerated noncitizens in a state measured by the Annual Survey of Jails. 

In [1]:
import pandas as pd
import numpy as np

sc_arrests = pd.read_excel('https://raw.githubusercontent.com/mgmills93/AEM_Spring_2019/master/enforcement/Secure_Communities_2017_Deportations.xlsx', header=4)
print(sc_arrests.sample(5))

      State    County Departed Date  \
77543    NM    Cibola    2016-07-16   
19780    NM  Sandoval    2016-06-17   
53277    TX    Brazos    2015-07-09   
87810    TX   Wharton    2015-01-07   
62694    TX    Reeves    2017-01-07   

                                  Port Of Departure Departed To Country  \
77543                           EAGLE PASS, TX, POE              MEXICO   
19780                              DEL RIO, TX, POE              MEXICO   
53277  NEWARK INTERNATIONAL AIRPORT, NEWARK NJ, POE               INDIA   
87810                              HOUSTON, TX, POE           GUATEMALA   
62694                           EAGLE PASS, TX, POE              MEXICO   

                                Case Status  \
77543  8-Excluded/Removed - Inadmissibility   
19780    6-Deported/Removed - Deportability   
53277    6-Deported/Removed - Deportability   
87810    6-Deported/Removed - Deportability   
62694  8-Excluded/Removed - Inadmissibility   

                               

In [0]:
county_state = sc_arrests.loc[:, 'State':'Departed Date']

###Below are completed rows State, County, and Arrest_Number

In [3]:
cs_group = county_state.groupby(['State', 'County']).count().reset_index()
cs_group.rename(columns={'Departed Date':'Arrest_Number'}, inplace=True)
cs_group['County_State'] = cs_group['County'].map(str) + ', ' + cs_group['State']
print(cs_group)

     State                     County  Arrest_Number  \
0       AK          Anchorage Borough              9   
1       AK  Matanuska-Susitna Borough              1   
2       AL                    Autauga              4   
3       AL                    Baldwin            145   
4       AL                    Barbour              2   
5       AL                     Blount              4   
6       AL                    Calhoun              7   
7       AL                   Cherokee              1   
8       AL                    Chilton             25   
9       AL                       Clay              1   
10      AL                   Cleburne              3   
11      AL                     Coffee              6   
12      AL                    Colbert              3   
13      AL                   Crenshaw              1   
14      AL                    Cullman             11   
15      AL                       Dale              3   
16      AL                     DeKalb           

### Below is the county-level hispanic population percentage for 2016

In [6]:
hisp_counties = pd.read_csv('https://raw.githubusercontent.com/mgmills93/AEM_Spring_2019/master/demographics/ACS_16_5YR_B03001_with_ann_copy.csv', encoding='latin-1', header=1)
geo_pop = hisp_counties.loc[:, 'Geography':'Estimate; Total: - Hispanic or Latino:']

new = geo_pop['Geography'].str.split(', ', n=1, expand=True)
geo_pop['County']=new[0]
geo_pop['State']=new[1]
geo_pop.drop(columns=['Geography'], inplace=True)

geo_pop['County'] = geo_pop['County'].map(lambda x: x.rstrip('County'))

geo_pop['Percent_Hispanic'] = geo_pop['Estimate; Total: - Hispanic or Latino:']/geo_pop['Estimate; Total:']*100
geo_pop.drop(columns=['Estimate; Total:', 'Estimate; Total: - Hispanic or Latino:'], inplace=True)

us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Colmbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',}

geo_pop['State']=geo_pop['State'].map(us_state_abbrev)
geo_pop = geo_pop.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
geo_pop['County_State'] = geo_pop['County'].map(str) + ', ' + geo_pop['State']

print(geo_pop.tail(10))



          County State  Percent_Hispanic    County_State
3132    Niobrara    WY          4.243395    Niobrara, WY
3133        Park    WY          5.618403        Park, WY
3134      Platte    WY          7.814645      Platte, WY
3135    Sheridan    WY          4.113755    Sheridan, WY
3136    Sublette    WY          7.814992    Sublette, WY
3137  Sweetwater    WY         15.859591  Sweetwater, WY
3138       Teton    WY         15.174822       Teton, WY
3139       Uinta    WY          8.959939       Uinta, WY
3140    Washakie    WY         13.962400    Washakie, WY
3141      Weston    WY          1.003484      Weston, WY


### Below are state-level noncitizen numbers from 2016




In [8]:
asj_data = pd.read_stata('https://raw.githubusercontent.com/mgmills93/AEM_Spring_2019/master/detention_centers/37135-0001-Data(2016ASJ_copy.dta')
list(asj_data)

asj_cols = asj_data.loc[:,['COUNTY', 'STATE', 'NONCITZ']]
#print(asj_cols.head(5))

asj_cols.drop(columns='COUNTY', inplace=True)
#print(asj_cols)
asj_cols.groupby('STATE').count().reset_index()


,STATE,NONCITZ
0,AL,25
1,AZ,8
2,AR,8
3,CA,65
4,CO,15
5,DC,1
6,FL,35
7,GA,74
8,ID,12
9,IL,15


###Below is the county-level population percentage that voted for trump in 2016

In [9]:
voting_data = pd.read_csv('https://raw.githubusercontent.com/tonmcg/US_County_Level_Election_Results_08-16/master/2016_US_County_Level_Presidential_Results.csv')
voting_cols = voting_data.loc[:,['per_gop', 'state_abbr', 'county_name']]
voting_cols.rename(columns={'county_name':'County', 'state_abbr':'State'}, inplace=True)
voting_cols['County'] = voting_cols['County'].map(lambda x: x.rstrip('County'))
voting_cols = voting_cols.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

voting_cols['County_State'] = voting_cols['County'].map(str) + ', ' + geo_pop['State']

print(voting_cols.tail(5))

       per_gop State      County    County_State
3136  0.729428    WY  Sweetwater  Sweetwater, WY
3137  0.321945    WY       Teton       Teton, WY
3138  0.764187    WY       Uinta       Uinta, WY
3139  0.783580    WY    Washakie    Washakie, WY
3140  0.869226    WY      Weston      Weston, WY


###Below I will merge for my final dataset

In [10]:
#merge arrests with hispanic data
cs_hisp = cs_group.merge(geo_pop, on='County_State', how='inner')
#print(cs_hisp.tail(10))
#merge cs_hisp with voting data
cs_hisp_vote = cs_hisp.merge(voting_cols, on='County_State', how='inner')
#print(cs_hisp_vote.tail(10))
cs_hisp_vote = cs_hisp_vote.loc[:, ['Arrest_Number', 'County_State', 'Percent_Hispanic', 'per_gop', 'State']]
print(cs_hisp_vote.sample(5))
#merge cs_hisp_vote and incarceration state data
def incarceration_row (row):
  if row ['State'] == 'AL' : 
    return '25'
  if row ['State'] == 'AZ' :
    return '8'
  if row ['State'] == 'AR' :
    return '8'
  if row ['State'] == 'CA' :
    return '65'
  if row ['State'] == 'CO' :
    return '15'
  if row ['State'] == 'DC' :
    return '1'
  if row ['State'] == 'FL' :
    return '35'
  if row ['State'] == 'GA' :
    return '74'
  if row ['State'] == 'ID' :
    return '12'
  if row ['State'] == 'IL' :
    return '15'
  if row ['State'] == 'IN' :
    return '24'
  if row ['State'] == 'IA' :
    return '11'
  if row ['State'] == 'KS' :
    return '13'
  if row ['State'] == 'KY' :
    return '28'
  if row ['State'] == 'LA' :
    return '38'
  if row ['State'] == 'ME' :
    return '2'
  if row ['State'] == 'MD' :
    return '13'
  if row ['State'] == 'MA' :
    return '11'
  if row ['State'] == 'MI' :
    return '18'
  if row ['State'] == 'MN' :
    return '21'
  if row ['State'] == 'MS' :
    return '25'
  if row ['State'] == 'MO' :
    return '18'
  if row ['State'] == 'MT' :
    return '4'
  if row ['State'] == 'NE' :
    return '6'
  if row ['State'] == 'NV' :
    return '7'
  if row ['State'] == 'NH' :
    return '3'
  if row ['State'] == 'NJ' :
    return '16'
  if row ['State'] == 'NM' :
    return '9'
  if row ['State'] == 'NY' :
    return '14'
  if row ['State'] == 'NC' :
    return '22'
  if row ['State'] == 'ND' :
    return '6'
  if row ['State'] == 'OH' :
    return '30'
  if row ['State'] == 'OK' :
    return '17'
  if row ['State'] == 'OR' :
    return '9'
  if row ['State'] == 'PA' :
    return '36'
  if row ['State'] == 'SC' :
    return '15'
  if row ['State'] == 'SD' :
    return '5'
  if row ['State'] == 'TN' :
    return '46'
  if row ['State'] == 'TX' :
    return '74'
  if row ['State'] == 'UT' :
    return '8'
  if row ['State'] == 'VA' :
    return '41'
  if row ['State'] == 'WA' :
    return '19'
  if row ['State'] == 'WV' :
    return '10'
  if row ['State'] == 'WI' :
    return '21'
  if row ['State'] == 'WY' :
    return '4'


cs_hisp_vote.apply(lambda row: incarceration_row(row), axis=1)

cs_hisp_vote['Incarceration_Number'] = cs_hisp_vote.apply (lambda row: incarceration_row(row), axis=1)
cs_hisp_vote.fillna(value=pd.np.nan, inplace=True)
cs_hisp_vote.fillna(method='ffill', inplace=True)



      Arrest_Number   County_State  Percent_Hispanic   per_gop State
1447              8  Sheboygan, WI          5.975588  0.555293    WI
368               1     Benton, IA          1.380249  0.705802    IN
1307             39  Tom Green, TX         38.062513  0.718455    TX
1351             36       Utah, UT         11.152351  0.513056    UT
1386              6     Orange, VA          4.209560  0.612636    VA


In [11]:
border_states = ['TX', 'AZ', 'NM', 'CA', 'AK', 'ME', 'NH', 'VT', 'MI', 'MN', 'ND', 'MT', 'ID', 'WA', 'NY']
cs_hisp_vote['Border'] = [1 if x in border_states else 0 for x in cs_hisp_vote['State']]
print(cs_hisp_vote.tail(5))
cs_hisp_vote.to_csv('regression_set.csv')

      Arrest_Number    County_State  Percent_Hispanic   per_gop State  \
1477              3    Sheridan, WY          4.113755  0.723620    WY   
1478              1    Sublette, WY          7.814992  0.793344    WY   
1479             26  Sweetwater, WY         15.859591  0.729428    WY   
1480             32       Teton, WY         15.174822  0.321945    WY   
1481              3       Uinta, WY          8.959939  0.764187    WY   

     Incarceration_Number  Border  
1477                    4       0  
1478                    4       0  
1479                    4       0  
1480                    4       0  
1481                    4       0  
